In [1]:
import pandas as pd
from gensim.parsing.preprocessing import preprocess_string
from gensim import corpora
import gensim
from gensim.models.coherencemodel import CoherenceModel
import re
# from tabulate import tabulate

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import os 
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import pickle

# import image module
from IPython.display import Image

In [2]:
# List stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
# print(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\E116189\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\E116189\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\E116189\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\E116189\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import random
# Set the global random seed
random.seed(42)
np.random.seed(42)

# Loading the files for a new run

In [6]:
df_aus = pd.read_csv('fwb-australia-topic-prediction.csv')

In [7]:
# Convert the timestamps to a proper datetime format
df_aus['publication_datetime'] = pd.to_datetime(df_aus['publication_date'], unit='ms')

# Retain just the date and drop the time
df_aus['publication_date'] = df_aus.publication_datetime.dt.date
df_aus

,text,region_of_origin,publication_date,publisher_name,preprocessed_sentence,preprocessed_list_with_bi_tri,preprocessed_len,topics,corrected_topics,revelant_topics,publication_datetime
0,"Forming a company key for one family HOW, wh...",AUSNZ AUSTR,2005-06-30,West Australian Newspapers Limited,"['forming', 'company', 'family', 'much', 'land...",['form' 'compani' 'famili' 'much' 'land' 'hand...,320,Topic 33,topic 33,NaN,2005-06-30
1,"There are many different types of homebuyers, ...",AUSNZ AUSTR,2017-04-29,Fairfax Media Management Pty Limited,"['many', 'different', 'types', 'homebuyers', '...",['mani' 'differ' 'type' 'homebuy' 'understand'...,68,Topic 31,topic 31,NaN,2017-04-29
2,Off-the-plan contracts review The NSW Governme...,AUSNZ AUSTR,2018-02-03,Fairfax Media Management Pty Limited,"['plan', 'contracts', 'review', 'government', ...",['plan' 'contract' 'review' 'govern' 'releas' ...,59,Topic 36,topic 36,NaN,2018-02-03
3,'Super must rise to 12%': MP Whitlam MP Stephe...,AUSNZ AUSTR,2019-10-09,Fairfax Media Management Pty Limited,"['super', 'must', 'rise', 'whitlam', 'stephen'...",['super' 'must' 'rise' 'whitlam' 'stephen_jon'...,130,Topic 28,topic 28,NaN,2019-10-09
4,Where there's a will TODAY I'd like to talk ab...,AUSNZ AUSTR,2019-05-25,Fairfax Media Management Pty Limited,"['today', 'like', 'talk', 'death', 'attention'...",['today' 'like' 'talk' 'death' 'attent' 'heave...,333,Topic 29,topic 23,topic 23,2019-05-25
...,...,...,...,...,...,...,...,...,...,...,...
119942,Super choice brings costs ONE in two small b...,AUSNZ AUSTR,2005-11-02,Nationwide News Pty Ltd.,"['super', 'choice', 'brings', 'costs', 'small'...",['super' 'choic' 'bring' 'cost' 'small' 'busi'...,42,Topic 47,topic 45,topic 45,2005-11-02
119943,"Home loans guide NO doc, low doc, combinatio...",AUSNZ AUSTR,2006-08-16,Nationwide News Pty Ltd.,"['home', 'loans', 'guide', 'combination', 'spl...",['home' 'loan' 'guid' 'combin' 'split' 'honeym...,82,Topic 40,topic 40,NaN,2006-08-16
119944,Taxing times for families TAXPAYERS having the...,AUSNZ AUSTR,2008-07-23,Nationwide News Pty. Ltd.,"['taxing', 'times', 'families', 'taxpayers', '...",['tax' 'time' 'famili' 'taxpay' 'return' 'prep...,112,Topic 49,topic 48,topic 48,2008-07-23
119945,Legal papers vital Legal papers vital W...,AUSNZ AUSTR,2007-09-05,Nationwide News Pty Ltd.,"['legal', 'papers', 'vital', 'legal', 'papers'...",['legal' 'paper' 'vital' 'legal' 'paper' 'vita...,158,Topic 45,topic 43,topic 43,2007-09-05


In [8]:
# Split the year and months
df_aus['publication_year'] = df_aus.publication_datetime.dt.year
df_aus['publication_month'] = df_aus.publication_datetime.dt.month

# prefix a 0 to the months
df_aus['publication_month'] = df_aus.publication_month.apply(lambda x: '0'+str(x) if len(str(x)) < 2 else str(x))

In [11]:
df_aus.publisher_name.value_counts()

Fairfax Media Management Pty Limited               40306
Nationwide News Pty Ltd.                           18854
News Ltd.                                          17957
Nationwide News Pty Ltd                             8996
Nationwide News Pty. Ltd.                           4586
Fairfax Digital Australia & New Zealand Limited     4572
APN Newspapers Pty Ltd                              4058
Federal Capital Press of Australia Pty Ltd          2643
F2 Australia & New Zealand Limited                  2085
HT Digital Streams Limited                          2048
F2 Australia & New Zealand Limited (Fairfax)        1981
News Ltd                                            1680
West Australian Newspapers Limited                  1607
Australian Broadcasting Corporation                 1263
Private Media Partners Pty Ltd.                      395
The Austrailian Securities Exchange Limited          394
The Conversation Media Group Limited                 394
News Bites Pty Ltd (Europe)    

In [160]:
df_grouped_df = df_aus.groupby(['corrected_topics', 'publication_year', 'publication_month']).count()[['text']].reset_index()

In [161]:
df_grouped_df

,corrected_topics,publication_year,publication_month,text
0,topic 1,1991,07,1
1,topic 1,1991,12,1
2,topic 1,1995,08,1
3,topic 1,1995,11,1
4,topic 1,1996,02,1
...,...,...,...,...
11322,topic 9,2022,05,4
11323,topic 9,2022,06,3
11324,topic 9,2022,07,3
11325,topic 9,2022,08,3


In [162]:
df_grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11327 entries, 0 to 11326
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   corrected_topics   11327 non-null  object
 1   publication_year   11327 non-null  int64 
 2   publication_month  11327 non-null  object
 3   text               11327 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 354.1+ KB


In [163]:
topic_names_inc_exc = pd.read_excel('Financial_wellbeing_topic_names.xlsx', sheet_name='in')

In [164]:
topic_names_inc_exc

,Topic Number,Article Count,Topic Name (created by research team),Included in Line Chart
0,25,11527,Banking,1.0
1,56,9697,Billionaires,NaN
2,51,7326,Start ups,1.0
3,40,7072,Criminal law,NaN
4,57,5779,Stock trading,1.0
5,26,4896,Pubilc statements (see 38),NaN
6,54,4759,Food and nutrition,NaN
7,21,3834,Social security and government benefits,1.0
8,31,3520,Trusts and lending,NaN
9,50,3230,Investment trusts,NaN


In [165]:
# Retain the appropriate topics only
imp_topics_df = topic_names_inc_exc[topic_names_inc_exc['Included in Line Chart'] == 1]

In [166]:
imp_topics_df

,Topic Number,Article Count,Topic Name (created by research team),Included in Line Chart
0,25,11527,Banking,1.0
2,51,7326,Start ups,1.0
4,57,5779,Stock trading,1.0
7,21,3834,Social security and government benefits,1.0
11,32,2955,Aged Care,1.0
12,45,2926,Property market,1.0
13,16,2816,Mortgages,1.0
14,43,2522,First homebuyers,1.0
15,35,2443,Financial scams,1.0
18,47,2366,Life insurance,1.0


In [167]:
imp_topics_df['Topic Number'] = imp_topics_df['Topic Number'].apply(lambda x: 'topic '+str(x))

c:\Users\E116189\RMIT\Topic-modelling\framework\tpm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [168]:
imp_topics_df

,Topic Number,Article Count,Topic Name (created by research team),Included in Line Chart
0,topic 25,11527,Banking,1.0
2,topic 51,7326,Start ups,1.0
4,topic 57,5779,Stock trading,1.0
7,topic 21,3834,Social security and government benefits,1.0
11,topic 32,2955,Aged Care,1.0
12,topic 45,2926,Property market,1.0
13,topic 16,2816,Mortgages,1.0
14,topic 43,2522,First homebuyers,1.0
15,topic 35,2443,Financial scams,1.0
18,topic 47,2366,Life insurance,1.0


In [169]:
topics_renamed = dict(zip(imp_topics_df['Topic Number'].values, imp_topics_df['Topic Name (created by research team)'].values))

In [170]:
topics_renamed

{'topic 25': 'Banking',
 'topic 51': 'Start ups',
 'topic 57': 'Stock trading',
 'topic 21': 'Social security and government benefits',
 'topic 32': 'Aged Care',
 'topic 45': 'Property market',
 'topic 16': 'Mortgages',
 'topic 43': 'First homebuyers',
 'topic 35': 'Financial scams',
 'topic 47': 'Life insurance',
 'topic 52': 'Retirement',
 'topic 49': 'Transport',
 'topic 29': 'Workplace gender equality',
 'topic 37': 'Health care',
 'topic 19': 'Superannuation',
 'topic 53': 'Generational change in housing affordabiliy',
 'topic 28': 'Art markets',
 'topic 23': 'Banking royal commission',
 'topic 46': 'Ethical investing',
 'topic 18': 'Executive remuneration'}

In [171]:
df_grouped_df

,corrected_topics,publication_year,publication_month,text
0,topic 1,1991,07,1
1,topic 1,1991,12,1
2,topic 1,1995,08,1
3,topic 1,1995,11,1
4,topic 1,1996,02,1
...,...,...,...,...
11322,topic 9,2022,05,4
11323,topic 9,2022,06,3
11324,topic 9,2022,07,3
11325,topic 9,2022,08,3


In [172]:
df_grouped_df['renamed_topics'] = df_grouped_df.corrected_topics.map(topics_renamed)

In [173]:
df_grouped_df = df_grouped_df.dropna()
df_grouped_df

,corrected_topics,publication_year,publication_month,text,renamed_topics
1256,topic 18,1995,05,1,Executive remuneration
1257,topic 18,1995,09,1,Executive remuneration
1258,topic 18,1995,11,1,Executive remuneration
1259,topic 18,1997,04,1,Executive remuneration
1260,topic 18,1997,08,1,Executive remuneration
...,...,...,...,...,...
9021,topic 52,2022,06,16,Retirement
9022,topic 52,2022,07,13,Retirement
9023,topic 52,2022,08,14,Retirement
9024,topic 52,2022,09,3,Retirement


In [174]:
# df_grouped_df = df_grouped_df[df_grouped_df.publication_year >= 2010]
df_grouped_df

,corrected_topics,publication_year,publication_month,text,renamed_topics
1256,topic 18,1995,05,1,Executive remuneration
1257,topic 18,1995,09,1,Executive remuneration
1258,topic 18,1995,11,1,Executive remuneration
1259,topic 18,1997,04,1,Executive remuneration
1260,topic 18,1997,08,1,Executive remuneration
...,...,...,...,...,...
9021,topic 52,2022,06,16,Retirement
9022,topic 52,2022,07,13,Retirement
9023,topic 52,2022,08,14,Retirement
9024,topic 52,2022,09,3,Retirement


In [175]:
df_grouped_df.renamed_topics.unique()

array(['Executive remuneration', 'Superannuation',
       'Social security and government benefits',
       'Banking royal commission', 'Art markets', 'Financial scams',
       'First homebuyers', 'Property market', 'Life insurance',
       'Start ups', 'Retirement'], dtype=object)

In [176]:
for i in df_grouped_df.renamed_topics.unique():
    print(f'if (name === "{i}")')
    sub_df = df_grouped_df[df_grouped_df.renamed_topics == i]
    # print(sub_df.publication_year.astype(str) + '-' + sub_df.publication_month)
    result = dict(zip(sub_df.publication_year.astype(str) + '-' + sub_df.publication_month, sub_df.text))
    result = [{'year': key, f'value{i}': value} for key, value in result.items()]

    print(f'data = {result}; \n')

if (name === "Executive remuneration")
data = [{'year': '1995-05', 'valueExecutive remuneration': 1}, {'year': '1995-09', 'valueExecutive remuneration': 1}, {'year': '1995-11', 'valueExecutive remuneration': 1}, {'year': '1997-04', 'valueExecutive remuneration': 1}, {'year': '1997-08', 'valueExecutive remuneration': 1}, {'year': '1997-12', 'valueExecutive remuneration': 1}, {'year': '1998-11', 'valueExecutive remuneration': 1}, {'year': '1999-01', 'valueExecutive remuneration': 2}, {'year': '1999-03', 'valueExecutive remuneration': 2}, {'year': '1999-06', 'valueExecutive remuneration': 1}, {'year': '1999-10', 'valueExecutive remuneration': 3}, {'year': '1999-11', 'valueExecutive remuneration': 1}, {'year': '2000-05', 'valueExecutive remuneration': 1}, {'year': '2000-06', 'valueExecutive remuneration': 1}, {'year': '2000-07', 'valueExecutive remuneration': 1}, {'year': '2001-01', 'valueExecutive remuneration': 2}, {'year': '2001-08', 'valueExecutive remuneration': 3}, {'year': '2001-09'